In [1]:
import numpy as np
from numpy.linalg import svd
from numpy.linalg import matrix_rank as rank
from numpy.linalg import inv

In [2]:
R_q = np.array([
    [0,  -1, 0,  0,  0,  0],
    [0,  0,  0,  0,  -1, 0],
    [0,  0,  -1, 0,  0,  0],
    [0,  0,  0,  0,  0,  -1],
    [0,  0,  0,  -1,  0,  0],
    [0,  1,  0,  0,  0,  0],
    [0,  0,  0,  0,  1,  0],
    [1,  0,  1,  0,  0,  0],
    [0,  0,  0,  0,  0,  1],
    [-1, 0,  0,  1,  0,  0]
], dtype=np.float)

R_p = np.array([
    [0,  1,  0,  0,  0,  0],
    [0,  0,  0,  0,  1,  0],
    [1,  0,  1,  0,  0,  0],
    [0,  0,  0,  0,  0,  1],
    [-1, 0,  0,  1,  0,  0],
    [0,  -1, 0,  0,  0,  0],
    [0,  0,  0,  0,  -1, 0],
    [0,  0,  -1, 0,  0,  0],
    [0,  0,  0,  0,  0,  -1],
    [0,  0,  0,  -1, 0,  0]
], dtype=np.float)

In [3]:
T_lb = np.array([
    [1,  0,  0,  0],
    [0,  1,  0,  0],
    [0,  0,  0,  0],
    [0,  0,  0,  0],
    [0,  0,  1,  0],
    [0,  0,  0,  0],
    [0,  0,  0,  0],
    [0,  0,  0,  1],
    [0,  0,  0,  0],
    [0,  0,  0,  0],
], dtype=np.float)

In [4]:
T_ulf = np.concatenate([T_lb, np.array([
    [0,  0],
    [0,  0],
    [0,  0],
    [-1, 0],
    [0,  -1],
    [0,  0],
    [0,  0],
    [0,  0],
    [1,  0],
    [0,  1],
], dtype=np.float)], axis=1)

In [5]:
T_blf = np.concatenate([T_ulf, np.array([
    [-1, 0,  1],
    [0,  0,  0],
    [0,  0,  0],
    [0,  -1, 0],
    [0,  -1, 0],
    [1,  1,  0],
    [0,  0,  0],
    [0,  0,  0],
    [0,  0,  -1],
    [0,  0,  -1],
], dtype=np.float)], axis=1)

In [6]:
P_q = np.array([
    [1,  1,  0,  0,  0],
    [-1, 0,  0,  0,  0],
    [0,  -1, 0,  0,  0],
    [0,  -1, 0,  0,  0],
    [-1, 0,  0,  0,  0],
    [0,  0,  0,  1,  1],
    [0,  0,  0,  -1, 0],
    [0,  0,  0,  0,  -1],
    [0,  0,  1,  0,  -1],
    [0,  0,  0,  -1, 0],
], dtype=np.float)

In [7]:
P_p = np.array([
    [1,  1,  0,  0,  0],
    [-1, 0,  0,  0,  0],
    [0,  -1, 0,  0,  0],
    [0,  -1, 0,  0,  0],
    [-1, 0,  1,  0,  0],
    [0,  0,  0,  1,  1],
    [0,  0,  0,  -1, 0],
    [0,  0,  0,  0,  -1],
    [0,  0,  0,  0,  -1],
    [0,  0,  0,  -1, 0],
], dtype=np.float)

In [8]:
S_lb_q = np.array([
    [0, 0],
    [1, 0],
    [0, 0],
    [0, 0],
    [0, -1],
    [0, 0],
    [0, 0],
    [0, 1],
    [0, 0],
    [0, 0]
], dtype=np.float)

S_lb_p = np.array([
    [0],
    [0],
    [0],
    [0],
    [1],
    [0],
    [0],
    [-1],
    [0],
    [0]
], dtype=np.float)

S_ulf_q = np.array([
    [0,  0,  0,  0],
    [0,  -1, 0,  0],
    [0,  0,  0,  0],
    [0,  0,  0,  -1],
    [0,  0,  -1, 0],
    [0,  0,  0,  0],
    [0,  1,  0,  0],
    [1,  0,  0,  0],
    [0,  0,  0,  1],
    [-1, 0,  1,  0]
], dtype=np.float)

S_ulf_p = np.array([
    [0,  0,  0],
    [-1, 0,  0],
    [1,  0,  0],
    [1,  1,  0],
    [0,  0,  1],
    [0,  0,  0],
    [0,  0,  0],
    [-1, 0,  0],
    [0,  -1, 0],
    [0,  0,  -1],
], dtype=np.float)

S_blf_p = np.array([
    [0,  0,  0,  0],
    [1,  0,  0,  0],
    [0,  1,  0,  0],
    [-1, 0,  0,  1],
    [0,  0,  1,  0],
    [0,  0,  0,  0],
    [0,  0,  0,  0],
    [0,  -1, 0,  0],
    [0,  0,  0,  -1],
    [0,  0,  -1, 0],
], dtype=np.float)

In [9]:
# Find minimum linearly independent basis
def find_basis(matrix):
    basis_index = []
    solution = 0
    previous_rank = 0
    for column in range(matrix.shape[1]):
        if previous_rank == 0:
            proposed_solution = matrix[:, column:column+1]
        else:
            proposed_solution = np.concatenate([solution, matrix[:, column:column+1]], axis=1)
        if np.linalg.matrix_rank(proposed_solution) > previous_rank:
            solution = proposed_solution
            basis_index.append(column)
            previous_rank += 1
    return solution, basis_index

def left_inverse(matrix):
    return np.dot(np.linalg.inv(np.dot(matrix.transpose(), matrix)), matrix.transpose())

In [10]:
def nullspace(A, atol=1e-13, rtol=0):
    """Compute an approximate basis for the nullspace of A.

    The algorithm used by this function is based on the singular value
    decomposition of `A`.

    Parameters
    ----------
    A : ndarray
        A should be at most 2-D.  A 1-D array with length k will be treated
        as a 2-D with shape (1, k)
    atol : float
        The absolute tolerance for a zero singular value.  Singular values
        smaller than `atol` are considered to be zero.
    rtol : float
        The relative tolerance.  Singular values less than rtol*smax are
        considered to be zero, where smax is the largest singular value.

    If both `atol` and `rtol` are positive, the combined tolerance is the
    maximum of the two; that is::
        tol = max(atol, rtol * smax)
    Singular values smaller than `tol` are considered to be zero.

    Return value
    ------------
    ns : ndarray
        If `A` is an array with shape (m, k), then `ns` will be an array
        with shape (k, n), where n is the estimated dimension of the
        nullspace of `A`.  The columns of `ns` are a basis for the
        nullspace; each element in numpy.dot(A, ns) will be approximately
        zero.
    """

    A = np.atleast_2d(A)
    u, s, vh = svd(A)
    tol = max(atol, rtol * s[0])
    nnz = (s >= tol).sum()
    ns = vh[nnz:].conj().T
    return ns

In [11]:
def normalize(matrix):
    new_matrix = np.copy(matrix)
    for col in range(matrix.shape[1]):
        items = matrix[:, col]
        min_item = np.min(np.abs([item for item in items if np.abs(item) > 1e-5]))
        new_matrix[:, col] /= min_item
    return np.round(new_matrix, decimals=4)

In [12]:
def compute_ranks(T_tract):
    print("Expectation over p")
    p_shape = T_lambdap.shape[1]
    basis, index = find_basis(nullspace(np.concatenate([T_lambdap, T_p, -T_tract], axis=1))[:p_shape, :])
    normalized = normalize(basis)
    print(normalized)
    print("Expectation over q")
    q_shape = T_lambdaq.shape[1]
    basis, index = find_basis(nullspace(np.concatenate([T_lambdaq, T_q, -T_tract], axis=1))[:q_shape, :])
    normalized = normalize(basis)
    print(normalized)

In [13]:
# print(rank(np.concatenate([T_lambdap, T_p], axis=1)))
# print(rank(np.concatenate([T_lambdaq, T_q], axis=1)))

# def compute_p_proj(matrix):
#     p_null = nullspace(P_p.transpose())
#     p_projection = np.dot(p_null, np.dot(np.linalg.inv(np.dot(p_null.transpose(), p_null)), p_null.transpose()))
#     proj = np.dot(p_projection, matrix)
    
#     basis, index = find_basis(proj)
#     normalized = normalize(basis)
#     return normalized
    
# def compute_q_proj(matrix):
#     q_null = nullspace(P_q.transpose())
#     q_projection = np.dot(q_null, np.dot(np.linalg.inv(np.dot(q_null.transpose(), q_null)), q_null.transpose()))
#     proj = np.dot(q_projection, matrix)

#     basis, index = find_basis(proj)
#     normalized = normalize(basis)
#     return normalized

def intersection(matrix1, matrix2):
    basis, index = find_basis(np.dot(matrix1, nullspace(np.concatenate([matrix1, -matrix2], axis=1))[:matrix1.shape[1], :]))
    return normalize(basis)
    
def union(matrix1, matrix2):
    return np.concatenate([matrix1, matrix2], axis=1)


# def compute_ranks_proj(T_tract):
#     p_null = nullspace(T_p.transpose())
#     p_projection = np.dot(p_null, np.dot(np.linalg.inv(np.dot(p_null.transpose(), p_null)), p_null.transpose()))
#     T_lambdap_proj = np.dot(p_projection, T_lambdaq)
    
#     q_null = nullspace(T_q.transpose())
#     q_projection = np.dot(q_null, np.dot(np.linalg.inv(np.dot(q_null.transpose(), q_null)), q_null.transpose()))
#     T_lambdaq_proj = np.dot(q_projection, T_lambdaq)

#     print("Expectation over p")
#     T_tract_proj = np.dot(p_projection, T_tract)
#     basis, index = find_basis(np.dot(T_lambdap_proj, nullspace(np.concatenate([T_lambdap_proj, -T_tract_proj], axis=1))[:T_lambdap_proj.shape[1], :]))
#     normalized = normalize(basis)
#     print(normalized)
#     print("Expectation over q")
#     T_tract_proj = np.dot(q_projection, T_tract)
#     basis, index = find_basis(np.dot(T_lambdaq_proj, nullspace(np.concatenate([T_lambdaq_proj, -T_tract_proj], axis=1))[:T_lambdaq_proj.shape[1], :]))
#     normalized = normalize(basis)
#     print(normalized)

In [14]:
print("Likelihood based")
print(rank(intersection(union(T_lb, P_p), union(R_p, P_p))), rank(intersection(union(T_lb, P_q), union(R_q, P_q))))
print(rank(union(S_lb_p, P_p)), rank(union(S_lb_q, P_q)))
print("Unary likelihood free")
print(rank(intersection(union(T_ulf, P_p), union(R_p, P_p))), rank(intersection(union(T_ulf, P_q), union(R_q, P_q))))
print(rank(union(S_ulf_p, P_p)), rank(union(S_ulf_q, P_q)))
print("Binary likelihood free")
print(rank(intersection(union(T_blf, P_p), union(R_p, P_p))), rank(intersection(union(T_blf, P_q), union(R_q, P_q))))
print(rank(union(S_blf_p, P_p)), rank(union(S_ulf_q, P_q)))
print("total")
print(rank(union(R_p, P_p)) + rank(union(R_q, P_q)))

Likelihood based
6 7
6 7
Unary likelihood free
8 9
8 9
Binary likelihood free
9 9
9 9
total
18
